In [1]:
import torch
import pandas as pd
from torch import nn
import pytorch_lightning as pl
import torch.utils.data as data_utils
import numpy as np
from os import cpu_count
from types import NoneType
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

/home/kacper/anaconda3/envs/gpu_torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import wandb
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint, LearningRateFinder
from pytorch_lightning.loggers import WandbLogger
from gensim.models import Word2Vec

In [3]:
torch.set_float32_matmul_precision("medium")

In [13]:
df=pd.read_parquet('./data/books.par')
df.columns

Index(['title', 'genre', 'summary', 'input_ids', 'att_mask', 'label',
       'mapped_inputs'],
      dtype='object')

# tweets dataset

In [6]:
df=pd.read_csv('./data/tweets.csv', names=['index','genre','summary']).drop(columns='index')

In [7]:
from transformers import GPT2Tokenizer, AutoTokenizer
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("openai-community/gpt2")
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token 
tokenized_summary=gpt2_tokenizer(df.summary.map(lambda x: x.strip().lower()).tolist(),
                    return_tensors='pt',
                    padding=True)
tokenized_summary['input_ids'].shape

torch.Size([149985, 835])

In [8]:
tokenized_summary['input_ids'][0,0].item()

38690

In [9]:
tokens_dict={}
for i, t in enumerate(tokenized_summary['input_ids'].unique()):
    tokens_dict[t.item()]=i

KeyboardInterrupt: 

In [ ]:
tokenized_summary['input_ids']=tokenized_summary['input_ids'].apply_(lambda x: tokens_dict[x])

In [ ]:
tokenized_summary['input_ids'][0,0].item()

21301

In [ ]:
tokenized_summary['attention_mask'].sum(axis=1).shape

torch.Size([149985])

In [ ]:
df.genre.unique()

array([0, 1])

In [ ]:
class TweetDataset(data_utils.Dataset):
    def __init__(self, tokenized_summary, labels, idxs):
        self.input_ids=tokenized_summary['input_ids'][idxs]
        self.lengths=tokenized_summary['attention_mask'].sum(axis=1)[idxs]
        self.y=torch.tensor(labels, dtype=torch.uint8)[idxs]
        self.no_classes=2
        self.max_len=self.input_ids.shape[1]

    def __len__(self):
        return self.y.shape[0]
    
    def __getitem__(self, indexes):
        return self.input_ids[indexes], self.lengths[indexes], self.y[indexes]
        

In [ ]:
train_idxs=torch.rand(df.shape[0])>0.8
val_idxs=~train_idxs

In [ ]:
train_data=TweetDataset(tokenized_summary, df.genre.values, train_idxs)
val_data=TweetDataset(tokenized_summary, df.genre.values, val_idxs)

train_dataloader=data_utils.DataLoader(train_data, batch_size=32, num_workers=cpu_count(),
                                       shuffle=True, drop_last=True)
val_dataloader=data_utils.DataLoader(val_data, batch_size=32, num_workers=cpu_count())

# dataset

In [5]:
class BookDataset(data_utils.Dataset):
    def __init__(self, df: pd.DataFrame):
        self.input_ids=torch.tensor(df.mapped_inputs, dtype=torch.int32)
        self.lengths=torch.tensor(df.att_mask.map(sum), dtype=torch.int32)
      #  self.att_masks=torch.tensor(df.att_mask, dtype=torch.float32)
        self.y=torch.tensor(df.label.values, dtype=torch.uint8)
        self.no_classes=df.label.nunique()
        self.max_len=self.input_ids.shape[1]

    def __len__(self):
        return self.y.shape[0]
    
    def __getitem__(self, indexes):
       # batch_max_len= torch.max(self.lengths[indexes])
        return self.input_ids[indexes], self.lengths[indexes], self.y[indexes]
        

In [ ]:
class DataSampler(data_utils.Sampler):
    def __init__(self, dataset, batch_size, shuffle=False, drop_last=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.drop_last = drop_last

    def __len__(self):
        return self.dataset.length

    def __iter__(self):
        order = np.arange(len(self))
        if self.shuffle:
            np.random.shuffle(order)
        if len(self) % self.batch_size:
            for i in range(0, len(self) - self.batch_size, self.batch_size):
                yield order[i : i + self.batch_size]
            if not self.drop_last:
                yield order[-(len(self) % self.batch_size) :]
        else:
            for i in range(0, len(self), self.batch_size):
                yield order[i : i + self.batch_size]


In [ ]:
class PackedDataset(data_utils.Dataset):
    def __init__(self, df: pd.DataFrame):
        self.input_ids=torch.tensor(df.mapped_inputs, dtype=torch.int32)
        self.lengths=torch.tensor(df.att_mask.map(sum), dtype=torch.int32)
        self.y=torch.tensor(df.label.values, dtype=torch.uint8)
        self.no_classes=df.label.nunique()
        self.max_len=self.input_ids.shape[1]

    def __len__(self):
        return self.y.shape[0]
    
    def __getitem__(self, indexes):
       
        return pack_padded_sequence(self.input_ids[indexes].view(-1, self.max_len), 
                                    self.lengths[indexes].view(-1), 
                                    batch_first=True, enforce_sorted=False), self.y[indexes]
        

In [ ]:
def simple_elementwise_apply(fn, packed_sequence):
    """applies a pointwise function fn to each element in packed_sequence"""
    return torch.nn.utils.rnn.PackedSequence(fn(packed_sequence.data), packed_sequence.batch_sizes)

In [ ]:
#pack_padded_sequence(emb(b[0]), b[1], batch_first=True, enforce_sorted=False)

In [ ]:
#simple_elementwise_apply(emb, val_data[:32][0])

In [6]:
df=df.iloc[np.random.permutation(df.shape[0])].reset_index(drop=True)
split=int(df.shape[0]*0.9)

train_df=df.iloc[:split]
val_df=df.iloc[split:].reset_index(drop=True)

In [7]:
_,label_weights=np.unique(train_df.label, return_counts=True)
label_weights=1/label_weights
label_weights=label_weights/np.sum(label_weights)
label_weights

array([0.02385822, 0.03267182, 0.04253915, 0.03501842, 0.03476234,
       0.02115128, 0.20895605, 0.19207071, 0.20228724, 0.20668478])

In [8]:
train_data=BookDataset(train_df)
val_data=BookDataset(val_df)

train_dataloader=data_utils.DataLoader(train_data, batch_size=32, num_workers=cpu_count(),
                                       shuffle=True, drop_last=True)
val_dataloader=data_utils.DataLoader(val_data, batch_size=32, num_workers=cpu_count())

/tmp/ipykernel_164841/4220535775.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  self.input_ids=torch.tensor(df.mapped_inputs, dtype=torch.int32)


# pl module

In [10]:
class BookGenreClassifier(pl.LightningModule):
    def __init__(self, model, lr=1e-2, loss=nn.CrossEntropyLoss(), l2=1e-5, lr_dc_step=3, lr_dc=0.1, **kwargs):
        super().__init__()
        self.save_hyperparameters(ignore=['model','loss'])
        if isinstance(loss.weight, NoneType):
            weighted_loss=False
        else: weighted_loss=True
        self.save_hyperparameters({'name':model.name, 
                                   'dropout_p':model.dropout_p,
                                   'w2v_init':model.w2v_init,
                                   'num_layers':model.num_layers,
                                   'nonlinearity':model.nonlinearity,
                                   'bidirectional':model.bidirectional,
                                   'weighted_loss':weighted_loss})
        self.lr=lr
        self.loss=loss
        self.model=model

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x = batch[:-1]
        y = batch[-1]
        logits=self(x)
        loss=self.loss(logits, y)

        self.log('train_loss', loss, prog_bar=True)
        return loss
    
    def evaluate(self, batch, mode=None):
        x = batch[:-1]
        y = batch[-1]
        logits=self(x)

        loss=self.loss(logits, y)

        preds=torch.argmax(logits, axis=1)
        acc=torch.sum(preds==y)/y.shape[0]
        # TODO add more metrics

        if mode:
            self.log(mode+'_loss', loss,  prog_bar=True)
            self.log(mode+'_acc', 100*acc,  prog_bar=True)

    def validation_step(self, batch, *args, **kwargs):
        return self.evaluate(batch, "val")
    def test_step(self, batch, *args, **kwargs):
        return self.evaluate(batch, "test")
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.model.parameters(), lr=self.lr, weight_decay=self.hparams.l2
        )
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            patience=self.hparams.lr_dc_step,
            factor=self.hparams.lr_dc,
            cooldown=1,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_acc",
                "strict": False,
                "interval": "epoch",
                "frequency": 1,
                "name": "scheduler_lr",
            },
        }

# different models

## dummy example

In [10]:
class DummyModel(nn.Module): 
    def __init__(self, in_dim=7031, hid_dim=128, out_dim=10):
        # dummy model as an example, just one hidden layer straight from list of tokens
        super().__init__()
        self.name='DummyModel'
        self.dropout_p=0
        self.l1=nn.Linear(in_dim, hid_dim)
        self.nonlinear=nn.Tanh()
        self.l2=nn.Linear(hid_dim, out_dim)

    def forward(self, x):
        in_ids, att_mask = x
        x=in_ids*att_mask
        x=self.l1(x)
        x=self.nonlinear(x)
        return self.l2(x)

### train

In [11]:
dm_model=BookGenreClassifier(DummyModel(), loss=nn.CrossEntropyLoss(weight=torch.tensor(label_weights, dtype=torch.float32)))

In [12]:
trainer=pl.Trainer(max_epochs=10)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/szymon/pythonvenvs/rocmwork/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [13]:
trainer.fit(dm_model, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | loss  | CrossEntropyLoss | 0      | train
1 | model | DummyModel       | 901 K  | train
---------------------------------------------------
901 K     Trainable params
0         Non-trainable params
901 K     Total params
3.606     Total estimated model params size (MB)


Sanity Checking: |                                                            | 0/? [00:00<?, ?it/s]

Training: |                                                                   | 0/? [00:00<?, ?it/s]

Validation: |                                                                 | 0/? [00:00<?, ?it/s]

Validation: |                                                                 | 0/? [00:00<?, ?it/s]

...

Validation: |                                                                 | 0/? [00:00<?, ?it/s]

Validation: |                                                                 | 0/? [00:00<?, ?it/s]

Validation: |                                                                 | 0/? [00:00<?, ?it/s]

Validation: |                                                                 | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


# global variables

In [15]:
train_data[0]

(tensor([  150,   333,   683,  ..., 26305, 26305, 26305], dtype=torch.int32),
 tensor(496, dtype=torch.int32),
 tensor(0, dtype=torch.uint8))

In [16]:
# for tweets
vocab_size=tokenized_summary['input_ids'].max()+1
bookwords = []
#for s in df.summary:
#    bookwords.append(list(tokenize(s, lowercase=True)))
for s, a in zip(tokenized_summary['input_ids'], tokenized_summary['attention_mask']):
    bookwords.append([str(i.item()) for i in s[a.to(bool)]]+[str(s[-1].item())])
print(bookwords[:1])


NameError: name 'tokenized_summary' is not defined

In [14]:
tokens_set=set()
for tokens in df.mapped_inputs.values:
    tokens_set=tokens_set.union(set(tokens))
vocab_size=len(tokens_set)
vocab_size

26306

In [15]:
bookwords = []
#for s in df.summary:
#    bookwords.append(list(tokenize(s, lowercase=True)))
for s in df.mapped_inputs:
    bookwords.append([str(i) for i in s])
print(bookwords[:1])

[['37', '1722', '165', '17871', '2821', '203', '151', '573', '21129', '1574', '151', '1799', '660', '1291', '375', '203', '204', '469', '8020', '221', '1468', '179', '9073', '151', '348', '173', '196', '14782', '13', '530', '2898', '236', '146', '313', '6585', '5114', '14', '1617', '834', '179', '13734', '293', '1075', '13772', '58', '13', '151', '1259', '4723', '477', '9089', '203', '7329', '469', '8020', '179', '351', '264', '8048', '151', '11527', '211', '18525', '17871', '2821', '1599', '539', '13', '469', '8020', '414', '447', '5227', '173', '3543', '634', '175', '151', '2047', '1490', '173', '333', '1038', '477', '222', '5644', '173', '3077', '145', '2723', '221', '9840', '7690', '13', '4936', '11', '151', '3543', '1775', '3468', '13738', '232', '1404', '236', '151', '590', '264', '12465', '173', '151', '3857', '4252', '175', '151', '1807', '13', '9089', '203', '23667', '1264', '287', '146', '1341', '3274', '232', '3276', '25614', '11', '1592', '236', '151', '5558', '174', '13617

In [16]:
del df
del train_df
del val_df

NameError: name 'train_df' is not defined

# w2v

In [58]:
class Word2VecSimple(nn.Module):
    def __init__(self, hid_dim, out_dim, vocab_size, dropout_p=0.5):
        super().__init__()
        self.name='Word2VecSimple'
        self.dropout_p=dropout_p
        self.nonlinearity='none'
        self.w2v_init=True
        self.num_layers=0
        self.bidirectional=False
        self.bimult=1+self.bidirectional

        w2vmodel = Word2Vec(bookwords, vector_size=hid_dim, min_count=0)
        self.emb = nn.Embedding(vocab_size, hid_dim, padding_idx=-1)

        emb_lst = []
        for v in range(vocab_size):
            emb_lst.append(w2vmodel.wv[str(v)])
        
        emb_mat = np.array(emb_lst)
        self.emb.load_state_dict({'weight': torch.from_numpy(emb_mat)})
        # load embeddings from pretrained word2vec
        #self.emb=nn.Embedding(vocab_size, hid_dim, padding_idx=-1)
        #self.out_layer=nn.Linear(hid_dim, out_dim)
        self.out_layer=nn.Sequential(nn.Dropout(p=dropout_p),
                                     nn.Linear(hid_dim, hid_dim),
                                     nn.Tanh(),
                                     nn.Dropout(p=0.2),
                                     nn.Linear(hid_dim, out_dim)
                                     )

    def forward(self, x):
        inputs, lengths = x

        w2v_output = self.emb(inputs)
        #avg_output = w2v_output.mean(dim=1)
        avg_output = torch.stack([w2v_output[i, :lengths[i]].mean(dim=0) for i in range(lengths.shape[0])])
       # print(f"w2v_output:{w2v_output.shape}")
        #print(f"avg_output:{avg_output.shape}")
        return self.out_layer(avg_output)

In [64]:
w2v_simple_model = BookGenreClassifier(Word2VecSimple(256, 10, vocab_size=vocab_size), 
                             loss=nn.CrossEntropyLoss(),#weight=torch.tensor(label_weights, dtype=torch.float32)),
                             lr_dc=0.1,
                             lr_dc_step=4,
                             )

In [65]:
wandb_logger = WandbLogger(
        project="ecoNLP", entity="kpuchalskixiv", log_model=False
    )

trainer=pl.Trainer(max_epochs=50,
                   callbacks=[
            EarlyStopping(
                monitor="val_acc", patience=10, mode="max", check_finite=True, check_on_train_epoch_end=False
            ),
            LearningRateMonitor(),
            ModelCheckpoint(monitor="val_acc", mode="max"),
         #   LearningRateFinder(min_lr=1e-4, num_training_steps=1000)
            ],
            logger=wandb_logger,
        )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [66]:
trainer.fit(w2v_simple_model, train_dataloader, val_dataloader)
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kpuchalskixiv. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | loss  | CrossEntropyLoss | 0     
1 | model | Word2VecSimple   | 6.6 M 
-------------------------------------------
6.6 M     Trainable params
0         Non-trainable params
6.6 M     Total params
26.222    Total estimated model params size (MB)


Epoch 2:  35%|███▍      | 321/930 [00:07<00:13, 45.84it/s, v_num=8woa, train_loss=0.767, val_loss=0.738, val_acc=50.20]

/home/kacper/anaconda3/envs/gpu_torch/lib/python3.12/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅█████
scheduler_lr,▁▁▁
train_loss,▂▁▂▃▇▄▂▂▄▃▃▄▃▃▅▃▃▃▄▂▂▅▃▅▁▅▅▅▄▁▁▂▂▄▂▁█▁▃▅
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
val_acc,▁█
val_loss,▁█
epoch,2
scheduler_lr,0.01
train_loss,0.83049
trainer/global_step,2149
val_acc,50.15308


# RNN

In [45]:
class SimpleRNN(nn.Module):
    def __init__(self, hid_dim, out_dim, vocab_size, dropout_p=0, nonlinearity='tanh', w2v_init=False):
        super().__init__()
        self.name='SimpleRNN'
        self.dropout_p=dropout_p
        self.nonlinearity=nonlinearity
        self.w2v_init=w2v_init
        # last token states 'end of string' and is repeated multiple time at the end of an input
        # therefore set its embedding to 0 with padding_idx=-1
        self.emb=nn.Embedding(vocab_size, hid_dim, padding_idx=-1) 
        if w2v_init:
            w2vmodel = Word2Vec(bookwords, vector_size=hid_dim, min_count=0)
            emb_lst = []
            for v in range(vocab_size):
                emb_lst.append(w2vmodel.wv[str(v)])
            emb_mat = np.array(emb_lst)
            self.emb.load_state_dict({'weight': torch.from_numpy(emb_mat)})
            
        self.model=nn.RNN(input_size=hid_dim, 
                          hidden_size=hid_dim, 
                          batch_first=True, 
                          dropout=dropout_p,
                         # bidirectional=True,
                          nonlinearity=nonlinearity)
        self.out_layer= nn.Linear(hid_dim, out_dim)
        #nn.Sequential(nn.Dropout(p=dropout_p),
         #                            nn.Linear(hid_dim, hid_dim),
          #                           nn.LeakyReLU(),
           #                          nn.Dropout(p=0.2),
            #                         nn.Linear(hid_dim, out_dim)
             #                        )

    def forward(self, x):
        inputs, att_mask=x
        rnn_input=self.emb(inputs)
        h0 = torch.randn(1, rnn_input.shape[0], rnn_input.shape[-1], device=rnn_input.device)
        hstates, hn = self.model(rnn_input, h0)
       # print(h_states.shape)
        hn[0]=torch.stack([hstates[e, int(i)-1] for e,i in enumerate(att_mask.sum(axis=1))])
        hn=hn.view(inputs.shape[0], -1)
        #use last hidden state as sequence representation
     #   hn=hn.squeeze()
        return self.out_layer(hn)

In [46]:
rnn_model=BookGenreClassifier(SimpleRNN(32, 10, vocab_size=vocab_size,
                                        nonlinearity='tanh',
                                        dropout_p=0.5,
                                        w2v_init=True,
                                        ), 
                             loss=nn.CrossEntropyLoss(),#weight=torch.tensor(label_weights, dtype=torch.float32),
                             lr=1e-3,
                             lr_dc=0.5,
                             lr_dc_step=4,
                             )

/home/kacper/anaconda3/envs/gpu_torch/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [47]:
wandb_logger = WandbLogger(
        project="ecoNLP", entity="kpuchalskixiv", log_model=True
    )

In [48]:
trainer=pl.Trainer(max_epochs=20,
                   callbacks=[
            EarlyStopping(
                monitor="val_acc", patience=10, mode="max", check_finite=True, check_on_train_epoch_end=False
            ),
            LearningRateMonitor(),
            ModelCheckpoint(monitor="val_acc", mode="max"),
         #   LearningRateFinder(min_lr=1e-4, num_training_steps=1000)
            ],
            logger=wandb_logger,
        )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [49]:
trainer.fit(rnn_model, train_dataloader, val_dataloader)
wandb.finish()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | loss  | CrossEntropyLoss | 0     
1 | model | SimpleRNN        | 1.2 M 
-------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.660     Total estimated model params size (MB)


Epoch 19: 100%|██████████| 130/130 [00:05<00:00, 22.49it/s, v_num=uo4k, train_loss=0.897, val_loss=2.350, val_acc=25.30]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 130/130 [00:05<00:00, 22.48it/s, v_num=uo4k, train_loss=0.897, val_loss=2.350, val_acc=25.30]


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
scheduler_lr,████████████▃▃▃▃▃▃▁▁
train_loss,█▇▇█▇█▇▇▇▇▆▇▆▆▆▇▆▆▅▅▃▅▄▄▄▃▃▃▃▄▄▂▃▂▁▁▂▂▂▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
val_acc,▁▃▄▁▃▄▅▅▆▆▄▆▅▆█▇▆▄▆▄
val_loss,▂▁▁▁▁▁▁▁▁▂▃▃▄▅▆▆▇▇██
epoch,19
scheduler_lr,0.00025
train_loss,0.89707
trainer/global_step,2599
val_acc,25.32189


In [24]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▂▂▂▃▃▃▄▄▄▅▅▆▆▆▁▁▂▂▂▃▃▃▃▄▄▅▅▅▆▆▆▆▇▇███
scheduler_lr,███████▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
train_loss,█▇▇███▇▇▆▆▆▆▆▅▅▄▅█▇███▇▆▅▇▄▄▃▄▃▂▂▂▂▂▁▁▂▁
trainer/global_step,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▁▁▂▂▂▂▃▃▄▄▄▄▅▅▅▆▆▆▇▇▇██
val_acc,▂▄▄▂▃▃▁▂▃▄▃▄▃▂▆▄▄▆▇▆█▅▆▆▄▅▅▄▄▃▃
val_loss,▁▁▁▁▁▁▂▂▂▃▃▃▄▁▁▁▁▁▁▁▂▃▄▄▅▆▆▆▇▇█
epoch,17
scheduler_lr,0.00025
train_loss,0.14059
trainer/global_step,2339
val_acc,22.103


# LSTM

In [80]:
class SimpleLSTM(nn.Module):
    def __init__(self, hid_dim, out_dim, vocab_size, dropout_p=0, nonlinearity='none', w2v_init=False, num_layers=1, bidirectional=False):
        super().__init__()
        self.name='SimpleLSTM'
        self.dropout_p=dropout_p
        self.nonlinearity=nonlinearity
        self.w2v_init=w2v_init
        self.num_layers=num_layers
        self.bidirectional=bidirectional
        self.bimult=1+bidirectional
        # last token states 'end of string' and is repeated multiple time at the end of an input
        # therefore set its embedding to 0 with padding_idx=-1
        self.emb=nn.Embedding(vocab_size, hid_dim, padding_idx=-1) 
        if w2v_init:
            w2vmodel = Word2Vec(bookwords, vector_size=hid_dim, min_count=0)
            emb_lst = []
            for v in range(vocab_size):
                emb_lst.append(w2vmodel.wv[str(v)])
            emb_mat = np.array(emb_lst)
            self.emb.load_state_dict({'weight': torch.from_numpy(emb_mat)})

        self.model=nn.LSTM(input_size=hid_dim, 
                           hidden_size=hid_dim, 
                           batch_first=True, 
                           dropout=dropout_p, 
                           num_layers=num_layers,
                           bidirectional=bidirectional)
      #  self.out_layer=nn.Linear(self.bimult*num_layers*hid_dim, out_dim)
        self.out_layer=nn.Sequential(nn.Dropout(p=dropout_p),
                                     nn.Linear(self.bimult*num_layers*hid_dim+hid_dim, hid_dim),
                                     nn.Tanh(),
                                     nn.Dropout(p=0.2),
                                     nn.Linear(hid_dim, out_dim)
                                     )

    def forward(self, x):
        inputs, lengths=x
        rnn_input=self.emb(inputs)

     #   avg_emb = torch.stack([rnn_input[i, :lengths[i]].mean(dim=0) for i in range(lengths.shape[0])])

        h0 = torch.randn(self.bimult*self.num_layers, rnn_input.shape[0], rnn_input.shape[-1], device=rnn_input.device)
        c0 = torch.randn(self.bimult*self.num_layers, rnn_input.shape[0], rnn_input.shape[-1], device=rnn_input.device)

        rnn_input=pack_padded_sequence(rnn_input,lengths.to('cpu').to(int), batch_first=True, enforce_sorted=False)
        hstates, (hn, _) = self.model(rnn_input, (h0, c0))
        padded_hstates, lengths=[x.to(inputs.device) for x in pad_packed_sequence(hstates, batch_first=True)]
        hstates_avg=padded_hstates.sum(dim=1).div(lengths.float().unsqueeze(dim=1))
        #return hstates
       # print(h_states.shape)
    #    hn[0]=torch.stack([hstates[e, int(i)-1] for e,i in enumerate(lengths)])
        hn=hn.view(inputs.shape[0], -1)
       # hn=hn.squeeze()
       #return self.out_layer(hn)
        return self.out_layer(torch.concat([hstates_avg, hn], dim=1))

In [83]:
lstm_model=BookGenreClassifier(SimpleLSTM(256, 10, vocab_size=vocab_size,
                                       # nonlinearity='tanh',
                                        dropout_p=0.5,
                                        w2v_init=True,
                                        num_layers=1,
                                        ), 
                             loss=nn.CrossEntropyLoss(),#weight=torch.tensor(label_weights, dtype=torch.float32)),
                             lr=1e-4,
                             lr_dc=0.5,
                             lr_dc_step=4,
                             )

/home/kacper/anaconda3/envs/gpu_torch/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [84]:
wandb_logger = WandbLogger(
        project="ecoNLP", entity="kpuchalskixiv", log_model=False
    )

In [85]:
trainer=pl.Trainer(max_epochs=50,
                   callbacks=[
            EarlyStopping(
                monitor="val_acc", patience=10, mode="max", check_finite=True, check_on_train_epoch_end=False
            ),
            LearningRateMonitor(),
            ModelCheckpoint(monitor="val_acc", mode="max"),
          #  LearningRateFinder(num_training_steps=1000)
            ],
            logger=wandb_logger,
        )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [86]:
trainer.fit(lstm_model, train_dataloader, val_dataloader)
wandb.finish()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | loss  | CrossEntropyLoss | 0     
1 | model | SimpleLSTM       | 7.4 M 
-------------------------------------------
7.4 M     Trainable params
0         Non-trainable params
7.4 M     Total params
29.578    Total estimated model params size (MB)


Epoch 25: 100%|██████████| 130/130 [00:18<00:00,  7.01it/s, v_num=q94r, train_loss=1.290, val_loss=1.470, val_acc=43.80]


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
scheduler_lr,██████▄▄▄▄▄▄▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train_loss,█▇▇▇▇▅▇█▄▅▇▃▃▅▃▄▅▄▆▂▂▄▅▃▆▃▄▁▃▃▁▄▂▄▄▃▁▃▃▃
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
val_acc,▁▃▄▄▅▅▆▆▆▇▇▆▆▇██▇▇▇▇▇▇████
val_loss,█▇▅▅▄▃▃▃▂▂▂▁▂▂▂▁▁▂▁▂▁▁▁▁▁▁
epoch,25
scheduler_lr,1e-05
train_loss,1.28331
trainer/global_step,3379
val_acc,43.77682


In [50]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


# Simple Transformer

In [17]:
class TransformerModel(nn.Module):
    def __init__(self, hid_dim, out_dim, vocab_size, dropout_p=0, nonlinearity='none', w2v_init=False, num_layers=1, bidirectional=False,
                 nheads=8,):
        super().__init__()
        self.name='SimpleTransformer'
        self.dropout_p=dropout_p
        self.nonlinearity=nonlinearity
        self.w2v_init=w2v_init
        self.num_layers=num_layers
        self.bidirectional=bidirectional
        self.nheads=nheads
        # last token states 'end of string' and is repeated multiple time at the end of an input
        # therefore set its embedding to 0 with padding_idx=-1
        self.emb=nn.Embedding(vocab_size, hid_dim, padding_idx=-1) 
        if w2v_init:
            w2vmodel = Word2Vec(bookwords, vector_size=hid_dim, min_count=0)
            emb_lst = []
            for v in range(vocab_size):
                emb_lst.append(w2vmodel.wv[str(v)])
            emb_mat = np.array(emb_lst)
            self.emb.load_state_dict({'weight': torch.from_numpy(emb_mat)})

        t_layer= nn.TransformerEncoderLayer(d_model=hid_dim, nhead=nheads, batch_first=True, dropout=dropout_p)
        self.model= nn.TransformerEncoder(t_layer, num_layers=num_layers)
      #  self.out_layer=nn.Linear(self.bimult*num_layers*hid_dim, out_dim)
        self.out_layer=nn.Sequential(nn.Dropout(p=dropout_p),
                                     nn.Linear(hid_dim+hid_dim, hid_dim),
                                     nn.Tanh(),
                                     nn.Dropout(p=0.2),
                                     nn.Linear(hid_dim, out_dim)
                                     )
        
    def forward(self, x):
        inputs, lengths=x
        max_len=min(512, torch.max(lengths))
        t_input=self.emb(inputs[:, :max_len])

        hstates=self.model(t_input)
        hstates_avg=hstates.sum(dim=1).div(lengths.float().unsqueeze(dim=1))
        hn=hstates[:,-1]

        return self.out_layer(torch.concat([hstates_avg, hn], dim=1))

In [18]:
lstm_model=BookGenreClassifier(TransformerModel(256, 10, vocab_size=vocab_size,
                                       # nonlinearity='tanh',
                                        dropout_p=0.5,
                                        w2v_init=True,
                                        num_layers=2,
                                        ), 
                             loss=nn.CrossEntropyLoss(),#weight=torch.tensor(label_weights, dtype=torch.float32)),
                             lr=1e-4,
                             lr_dc=0.5,
                             lr_dc_step=4,
                             )

In [19]:
wandb_logger = WandbLogger(
        project="ecoNLP", entity="kpuchalskixiv", log_model=False
    )

In [20]:
trainer=pl.Trainer(max_epochs=50,
                   callbacks=[
            EarlyStopping(
                monitor="val_acc", patience=10, mode="max", check_finite=True, check_on_train_epoch_end=False
            ),
            LearningRateMonitor(),
            ModelCheckpoint(monitor="val_acc", mode="max"),
          #  LearningRateFinder(num_training_steps=1000)
            ],
            logger=wandb_logger,
        )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [21]:
trainer.fit(lstm_model, train_dataloader, val_dataloader)
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kpuchalskixiv. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | loss  | CrossEntropyLoss | 0     
1 | model | TransformerModel | 9.5 M 
-------------------------------------------
9.5 M     Trainable params
0         Non-trainable params
9.5 M     Total params
37.994    Total estimated model params size (MB)


Epoch 4:  30%|███       | 39/130 [00:04<00:10,  8.72it/s, v_num=hev7, train_loss=1.510, val_loss=1.760, val_acc=31.50] 